<a href="https://colab.research.google.com/github/thewildox/Monte-Carlo-Simulation-for-Portfolio-Returns/blob/main/GBM_and_Monte_Carlo_Sims.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [ ]:
T = 1.0
N = 252
dt = T / N
n_simulations = 10000
ticker = "^GSPC"
start_date = "2015-01-01"
end_date = "2025-01-01"

In [ ]:
data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)
prices = data["Close"].squeeze()
prices.head()

In [ ]:
log_returns = np.log(prices / prices.shift(1)).dropna()
mu = float(log_returns.mean()) * 252
sigma = float(log_returns.std()) * np.sqrt(252)

print(f"Estimated Drift (mu): {mu:.4f}")
print(f"Estimated Volatility (sigma): {sigma:.4f}")

In [ ]:
S0 = float(prices.iloc[-1])

Z = np.random.standard_normal((N, n_simulations))

drift = (mu - 0.5 * sigma**2) * dt
diffusion = sigma * np.sqrt(dt) * Z

price_paths = np.zeros((N + 1, n_simulations))
price_paths[0] = S0
price_paths[1:] = S0 * np.exp(np.cumsum(drift + diffusion, axis=0))

In [ ]:
final_prices = price_paths[-1]

expected_value = final_prices.mean()
median_value = np.median(final_prices)
prob_loss = np.mean(final_prices < S0)
var_5 = np.percentile(final_prices, 5)

print("\nMonte Carlo Results:")
print(f"Expected Final Value: {expected_value:.2f}")
print(f"Median Final Value: {median_value:.2f}")
print(f"Probability of Loss: {prob_loss:.2%}")
print(f"5% Value-at-Risk (VaR): {var_5:.2f}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(price_paths[:, :100])
plt.title("Simulated Price Paths (First 100)")
plt.xlabel("Time Steps")
plt.ylabel("Price")
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(final_prices, bins=50, density=True)
plt.title("Distribution of Final Portfolio Values")
plt.xlabel("Final Price")
plt.ylabel("Density")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sorted_prices = np.sort(final_prices)
cdf = np.arange(len(sorted_prices)) / len(sorted_prices)
plt.plot(sorted_prices, cdf)
plt.title("CDF of Final Portfolio Values")
plt.xlabel("Final Price")
plt.ylabel("Probability")
plt.show()
